In [1]:
# %run 2.decoder.ipynb

# import torch
# from transformers import WhisperForConditionalGeneration

# pretrained = WhisperForConditionalGeneration.from_pretrained(
#     'openai/whisper-small').model.decoder
# decoder = load_decoder(pretrained)

In [2]:
def forward_atten(self, x, cache_kv):
    q = self.q(x).reshape(1, 1, 12, 64).transpose(1, 2) * 0.125
    k = self.k(x).reshape(1, -1, 12, 64).transpose(1, 2)
    v = self.v(x).reshape(1, -1, 12, 64).transpose(1, 2)

    if cache_kv:
        k = torch.cat([cache_kv[0], k], dim=2)
        v = torch.cat([cache_kv[1], v], dim=2)

    cache_kv = k, v

    q = q.reshape(12, -1, 64)
    k = k.reshape(12, -1, 64)
    v = v.reshape(12, -1, 64)

    atten = q.bmm(k.transpose(1, 2)).softmax(dim=-1).bmm(v)

    atten = atten.reshape(1, 12, 1, 64).transpose(1, 2).reshape(1, 1, 768)

    atten = self.out(atten)

    return atten, cache_kv


# x = torch.randn(1, 1, 768)
# cache_kv = torch.randn(1, 12, 2, 64), torch.randn(1, 12, 2, 64)

# out1, (k1, v1) = forward_atten(decoder.layer[0].atten, x, cache_kv)
# out2, _, (k2, v2) = pretrained.layers[0].self_attn(x, past_key_value=cache_kv)

# (out1 == out2).all(), (k1 == k2).all(), (v1 == v2).all()

In [3]:
def forward_cross_atten(self, x, kv, cache_kv):
    q = self.q(x).reshape(1, 1, 12, 64).transpose(1, 2) * 0.125
    if cache_kv:
        k, v = cache_kv
    else:
        k = self.k(kv).reshape(1, -1, 12, 64).transpose(1, 2)
        v = self.v(kv).reshape(1, -1, 12, 64).transpose(1, 2)
        cache_kv = k, v

    q = q.reshape(12, -1, 64)
    k = k.reshape(12, -1, 64)
    v = v.reshape(12, -1, 64)

    atten = q.bmm(k.transpose(1, 2)).softmax(dim=-1).bmm(v).reshape(
        1, 12, 1, 64).transpose(1, 2).reshape(1, 1, 768)

    atten = self.out(atten)

    return atten, cache_kv


# x = torch.randn(1, 1, 768)
# kv = torch.randn(1, 1500, 768)
# cache_kv = torch.randn(1, 12, 1500, 64), torch.randn(1, 12, 1500, 64)

# out1, (k1, v1) = forward_cross_atten(decoder.layer[0].cross_atten, x, kv,
#                                      cache_kv)
# out2, _, (k2, v2) = pretrained.layers[0].encoder_attn(x, kv, cache_kv)

# (out1 == out2).all(), (k1 == k2).all(), (v1 == v2).all()

In [4]:
def forward_layer(self, x, kv, cache_kv):
    next_cache_kv = ()

    res = x
    x, _cache = forward_atten(self.atten,
                              self.norm1(x),
                              cache_kv=cache_kv[:2] if cache_kv else None)
    x = x + res
    next_cache_kv += _cache

    res = x
    x, _cache = forward_cross_atten(
        self.cross_atten,
        self.norm2(x),
        kv,
        cache_kv=cache_kv[2:] if cache_kv else None)
    x = x + res
    next_cache_kv += _cache

    return self.s(x) + x, next_cache_kv


# x = torch.randn(1, 1, 768)
# kv = torch.randn(1, 1500, 768)
# cache_kv = torch.randn(1, 12, 2, 64), torch.randn(1, 12, 2, 64), torch.randn(
#     1, 12, 1500, 64), torch.randn(1, 12, 1500, 64)

# out1, (k1, v1, ck1, cv1) = forward_layer(decoder.layer[0], x, kv, cache_kv)
# out2, (k2, v2, ck2, cv2) = pretrained.layers[0](x,
#                                                 encoder_hidden_states=kv,
#                                                 past_key_value=cache_kv)

# (out1 == out2).all(), (k1 == k2).all(), (v1 == v2).all(), (ck1 == ck2).all(), (
#     cv1 == cv2).all()

In [5]:
def forward_decoder(self, x, kv, cache_kv):
    pos_offset = cache_kv[0][0].shape[2] if cache_kv else 0

    x = self.embed(x) + self.embed_pos.weight[pos_offset:pos_offset +
                                              x.shape[1]]

    next_cache_kv = []
    for i, layer in enumerate(self.layer):
        x, _cache = forward_layer(layer, x, kv,
                                  cache_kv[i] if cache_kv else None)
        next_cache_kv.append(_cache)

    return self.norm(x), tuple(next_cache_kv)


# x = torch.LongTensor([[95]])
# kv = torch.randn(1, 1500, 768)

# cache_kv = [(torch.randn(1, 12, 2, 64), torch.randn(1, 12, 2, 64),
#              torch.randn(1, 12, 1500, 64), torch.randn(1, 12, 1500, 64))] * 12
# cache_kv = tuple(cache_kv)

# out1, cache1 = forward_decoder(decoder, x, kv, cache_kv)
# out2 = pretrained(x,
#                   encoder_hidden_states=kv,
#                   past_key_values=cache_kv,
#                   use_cache=True,
#                   output_attentions=False,
#                   output_hidden_states=False,
#                   return_dict=True)

# for i, j in zip(cache1, out2.past_key_values):
#     for t in range(4):
#         print(t, (i[t] == j[t]).all())

# (out1 == out2.last_hidden_state).all()